# Clustering Addresses on the Bitcoin Blockchain

### Dimitris Tsementzis

In [ ]:
import requests
import time
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
import json

The aim of this notebook is to explain a method for clustering addresses on the Bitcoin blockchain, and indicate a use case for this information.

It accompanies the **blocktxviz** application available <a href="https://blocktxviz.herokuapp.com">here</a>.

By "clustering" here we do not mean the idea of grouping addresses on the Bitcoin blockchain to "users" to which they belong in the real world (as e.g. is explored in <a href="https://cseweb.ucsd.edu/~smeiklejohn/files/imc13.pdf">this</a> and <a href="https://eprint.iacr.org/2012/584.pdf">this</a> paper), but rather the idea of clustering addresses according to their behaviour **only within** the Bitcoin blockchain. 

Contrary to many of the approaches that we have seen, this information seems to us to be far more relevant for predicting the financial behaviour (e.g. USD value) of Bitcoin. 

Furthermore it generalizes more easily to other cryptocurrencies, rather than relying on, say, Bitcoin-specific heuristics.

## Obtaining the data

Data for transaction addresses was obtained from <a href="http://blockchain.info">Blockchain.info</a> in the form of JSON files.

This is done as follows.

First, we define the classes `BitcoinBlock` and `BitcoinAddress` tailored to the data as it is made available on the <a href="http://blockchain.info">Blockchain.info</a> API.

In [ ]:
from addrstats import BitcoinBlock, BitcoinAddress

The following script will then sample addresses that appear on recently minted blocks. The parameters are as follows:

* `dayrange` is the range of days, starting from the present, from which blocks will be pooled
* `blocksampleparameter` is the fraction of blocks from the pool that we will get addresses from
* `addrsampleparameter` is the fraction of addresses from each block that we will get data for

(Note that it is also possible to give integers for sampling parameters, to give an exact number for the samples. This requires changing the `frac` keyword in the `sample` method to `n` (e.g. `[...].sample(n = blocksampleparameter`.)

In [ ]:
dayrange = 1
blocksampleparameter = 0.01
addrsampleparameter = 0.01

Firstly, we obtain the blockhashes, for our given range.

In [ ]:
days = [int(round(int(round(time.time() * 1000)) - (n * 8.64e+7))) for n in range(dayrange)]
blockhashes = {}
for day in days:
    try:
        last24hourblocks = requests.get('https://blockchain.info/blocks/'+str(day)+'?format=json').json()
        last24_blocklist = [x['hash'] for x in last24hourblocks['blocks']]
        blockhashes[day] = list(pd.DataFrame({'A':last24_blocklist})['A'].sample(frac = blocksampleparameter))
    except:
        print('Could not obtain blocks for '+str(day))

sample = [x for y in blockhashes.values() for x in y]

In [ ]:
#Check how many blocks are in our sample
len(sample)

Now we sample addresses from these blocks. 

**A note of caution:** Before running the cell below note that this is potentially **a very large amount of data**. For `dayrange = 90` and the sampling parameters set to $0.1$ and $0.01$ the below cell will scrape approximately **70 GB** of data. So in order to test it out, set really low numbers. Alternatively, you can comment out the `json.dump` commands which will prevent the files getting saved to your local directory (the scraping will still take a very long time). 

The aim of the script below is to obtain statistics directly for each address sample from the blocks in our sample (`addr_stats`). The thought behind this is to make this analysis independent of actually having a database available. Practically, however, one would like to query directly from a database, especially if one is to make real-time use of this information.

To give an indication of what to expect, if the sample contains only 3 blocks, the below script will run for approximately 5 minutes.

In [ ]:
addr_stats = []
for blockhash in sample:
    print('Scouring '+blockhash+ ' for addresses...')
    try:
        b = BitcoinBlock(blockhash)
        with open('block_'+blockhash+'.json','w') as f:
            json.dump(b.block,f)
        try:
            A = b.get_addresses()
            for addr in list(pd.DataFrame(A).sample(frac = addrsampleparameter)[0]):
                print('Getting data for '+addr+' ...')
                try:
                    a = BitcoinAddress(addr)
                    with open('address_'+a.addrkey+'.json','w') as f:
                        json.dump(a.address, f)  
                    addr_stats.append(BitcoinAddress(addr).stats())
                except:
                    print('...failed')
        except:
            print('...failed')
    except:
        print('...failed to obtain block')

We can now write the obtained statistics in a text file (as space separated columns) for further analysis.

In [ ]:
with open('stats.txt','w') as f:
    f.write('\n'.join([' '.join(map(str,x)) for x in addr_stats]))

Finally, as an aside, if one is interested only in the blocks, the following code will save all the blocks in the `sample` (but not any addresses on them)

In [ ]:
for blockhash in blocks:
    print('Getting data for '+blockhash+ ' ...')
    try:
        b = BitcoinBlock(blockhash)
        with open('block_'+blockhash+'.json','w') as f:
            json.dump(b.block,f)
    except:
        print('...failed')

## Analyzing the statistics for the addresses

Let us now load some pre-computed statistics that have been obtained over large samples using the code above.

(If you wish to use the data obtained above, use `training = 'stats.txt'` instead. However, the rest of the analysis may not apply, although, of course, we do expect it to apply to any statistics obtained from a large enough sample of addresses.)

In [ ]:
training = 'stats_addrsample.txt'
df = pd.read_table(training, delim_whitespace=True, header = None)
df.columns = ['Total Transactions','BTC Received','BTC Sent','Total Senders','Total Recipients','Avg Tx Freq',
                     'Max Tx interval','Min Tx interval']

Let us get a sense of what the data looks like...

In [ ]:
df.head()

...its size...

In [ ]:
df.shape

...and some summary statistics of the features

In [ ]:
df.describe()

Some things to note here are the following.

Firstly, the overwhelming majority of addresses have been sent BTC by and received BTC from a small number of addresses.

In [ ]:
plt.show(df['Total Recipients'].hist(bins = 100, alpha = 0.8))
plt.show(df['Total Senders'].hist(bins = 100, alpha = 0.8))

In [ ]:
# A function to plot histograms for Total Recipients and Total Senders in the range [n_min,n_max]
def hist_rec_send(n_min,n_max):
    plt.show(df[(df['Total Recipients'] > n_min) & (df['Total Recipients'] < n_max)]['Total Recipients']\
         .hist(bins = 100, alpha = 0.6))
    plt.show(df[(df['Total Senders'] > n_min) & (df['Total Senders'] < n_max)]['Total Senders']\
         .hist(bins = 100, alpha = 0.6))

Of particular interest is the spike in number of recipients around the 3000-5000 range, and the fact that no signifcant such spike exists in the number of senders:

In [ ]:
hist_rec_send(1000,20000)

And we also notice very similar behavior with respect to the distribution of the amount of BTC sent and received by addresses:

In [ ]:
# A function to plot histograms for Total Recipients and Total Senders in the range [n_min,n_max]
def hist_BTC_rec_send(n_min,n_max):
    plt.show(df[(df['BTC Received'] > n_min) & (df['BTC Received'] < n_max)]['BTC Received']\
         .hist(bins = 100, alpha = 0.6))
    plt.show(df[(df['BTC Sent'] > n_min) & (df['BTC Sent'] < n_max)]['BTC Sent']\
         .hist(bins = 100, alpha = 0.6))

In [ ]:
hist_BTC_rec_send(0.0,50000.0)

Overall, the irregularities in the features provide some confidence that there are significantly distinct addresses appearing on the BTC blockchain.

It is therefore reasonable to investigate meaningful ways in which to cluster them.

## Clustering the Addresses

We will now cluster the addresses using KMeans clustering.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

To figure out the number of clusters $k$ we run an "elbow test".

An elbow test consist in ploting inertia (the sum of distances of each point from its cluster center) against $k$.

The "elbow point" is the value of $k$ at which the inertia begins decreasing at a diminishing rate.

It is a good indication of a "correct" or "reasonable" value for $k$.

In [ ]:
#we rescale our data first
df_scaled = StandardScaler().fit_transform(df)

In [ ]:
# a function that returns the elbow plot for k in the range [minclusters,maxclusters]
def elbowplot(df_scaled,minclusters,maxclusters):
    inertiae = []
    for n in range(minclusters,maxclusters):
        model = KMeans(n_clusters = n,random_state=63).fit(df)
        inertiae.append((n,model.inertia_))
    dftoplot = pd.DataFrame(inertiae,columns=['clusters','inertia'])
    return dftoplot.plot(x='clusters',y='inertia',style='-o')

In [ ]:
elbowplot(df_scaled,5,30)

Here are also a few more pre-computed elbow plots, on several variations of both the address sample and the features selected.

![Elbow Plot](Elbow_2.png)
![Elbow Plot](Elbow_3.png)
![Elbow Plot](Elbow_5.png)

Another metric we can use (in the absence of acceptable ground truth values) is the sillhouette coefiicient.

The silhouette coefficient $\alpha$ ranges inside $[-1,1]$ and scores closest to $1$ indicate good clustering.

Unfortunately, the results seem rather inconclusive, as the plot generated by the two cells below would indicate, although it does show a preference for a much smaller number of clusters (which could be the result of certain biases). 

Furthermore, due to computational limitations, we are also sampling 10% of the data in calculating the silhouette scores below, which could present another source of bias, as repeated runs of the below code demonstrate significant changes in the shape of the plot.

In [ ]:
from sklearn import metrics

def silhouetteplot(df,minclusters,maxclusters):
    scores = []
    for n in range(minclusters,maxclusters):
        X = pd.DataFrame(df).sample(frac=0.1)
        model = KMeans(n_clusters = n,random_state=63).fit(X)
        y = model.predict(X)
        scores.append((n,metrics.silhouette_score(X, y,metric='euclidean')))
    dftoplot = pd.DataFrame(scores,columns=['clusters','score'])
    return dftoplot.plot(x='clusters',y='score',style='-o')

In [ ]:
silhouetteplot(df_scaled,3,30) 

Thus, disregrading the silhouette score, and going solely on the elbow tests, the number $k = 20$ seems to us a reasonable one. 

We can of course vary $k$ at will, and even use it as a hyperparameter if a further model is built on the cluster data, but having some notion of a reasonable "number of address types" on the BTC blockchain seems to us of intrinsic interest.

So we set our model at $k = 20$.

In [ ]:
addrtype = Pipeline([('StandardScaler',StandardScaler()),
                  ('KMeans',KMeans(n_clusters = 20, random_state=33))])

In [ ]:
addrtype.fit(df)

## Visualizing Blocks Based on Address Activity

We can now use our model `addrtype` to visualize the activity on individual blocks in the blockchain.

First we can pick a block (by default the latest block, retrieved from the Blockchain.info API)...

In [ ]:
import requests

In [ ]:
block = requests.get('https://blockchain.info/q/latesthash')
b = BitcoinBlock(block.text)

...get the addresses on it...

In [ ]:
blockaddresses = b.get_addresses()

...see how many of them there are...

In [ ]:
len(blockaddresses)

...and also record the value of the transactions they were involved in.

In [ ]:
blockaddrval = b.get_addrval()
txbal = blockaddrval[0]
txapp = blockaddrval[1]

Now, we obtain, for each address, its statistics...

In [ ]:
A = []

...setting a sampling parameter and obtaining a sample...

In [ ]:
sampleparam = 500
sample = pd.DataFrame(blockaddresses).sample(n=sampleparam)[0].tolist()

...since even for `sampleparam = 500` the below computation will take approximately 5-10 minutes:

In [ ]:
for a in sample:
    print('Getting data for ', a,'...')
    try:
        A.append((a,txapp[a],txbal[a],BitcoinAddress(a).stats()))
        print('...success')
    except:
        print('...failed')

The below cells will serialize the statistics for the block above, in case one needs it for reuse.

In [ ]:
import pickle

In [ ]:
#save the data for re-use
pickle.dump(A,open('stats_'+block.text,'wb'))

In [ ]:
C = [(a[0],a[1],a[2],addrtype.predict([a[3]])[0]) for a in A]
dfb = pd.DataFrame(C,columns=['address','appearances','balance','cluster'])

In [ ]:
dftoplot = dfb.groupby('cluster').sum()
dftoplot['BTC'] = dftoplot['balance'] * 0.00000001
dftoplot = dftoplot[['appearances','BTC']]

In [ ]:
dftoplot

In [ ]:
xs = dftoplot.index
ys = np.log(dftoplot['appearances'])
btcvalue = dftoplot['BTC'].values
data_normalizer = mp.colors.Normalize()
color_map = mp.colors.LinearSegmentedColormap("my_map",
            {'blue':[(0.0,  1.0, 1.0),(0.5,  0.5, 0.5),(1.0,  0.0, 0.0)],\
             'green': [(0.0,  0.0, 0.0),(0.25, 1.0, 1.0),(0.75, 0.0, 0.0),(1.0,  0.0, 0.0)],\
             'red':  [(0.0,  0.0, 0.0),(0.5,  0.5, 0.5),(1.0,  1.0, 1.0)]})
# Map xs to numbers:
N = len(xs)
x_nums = np.arange(1, N+1)
# Plot a bar graph:
plt.bar(x_nums,ys,align="center",color = color_map(data_normalizer(btcvalue)))
# Change x numbers back to strings:
plt.xticks(x_nums, xs)
plt.figure(figsize=(200,50))

The above bar chart can be thought of as the signature of a particular block. 

Bar colors indicate net inflow/outflow of BTC for a given cluster of addresses relative to the rest of the clusters.

For example, a blue bar (corresponding to a cluster $n$, say) indicates that the addresses belonging to cluster $n$ sent more BTC than they received and had more outflow than the rest of the clusters. 

Let us now set things up so that we can plot for multiple values of $k$.

To speed things up (and to facilitate parallelization later) let's memoize the clustering models (this may take around 5-10 minutes).

In [ ]:
addrtypemodel = {}
for k in range(2,60):
    print('Computing clustering for k='+str(k)+'...')
    addrtype = Pipeline([('StandardScaler',StandardScaler()),
                  ('KMeans',KMeans(n_clusters = k, random_state=33))])
    addrtype.fit(df)
    addrtypemodel[k] = addrtype
    print('...done')

In [ ]:
# serialize for re-use, if needed -- commented out to prevent overwrites.
#pickle.dump(addrtypemodel,open('addrtypemodel.pkl','wb'))

In [ ]:
addrtypemodel = pickle.load(open('addrtypemodel_upto_60.pkl','rb'))

And we now put everything together in a function which returns the signature of a block for a given $k$...

In [ ]:
def blocksignature(block,k):
    C = [(a[0],a[1],a[2],addrtypemodel[k].predict([a[3]])[0]) for a in block]
    dfb = pd.DataFrame(C,columns=['address','appearances','balance','cluster'])
    dftoplot = dfb.groupby('cluster').sum()
    dftoplot['BTC'] = dftoplot['balance'] * 0.00000001
    dftoplot = dftoplot[['appearances','BTC']]
    xs = dftoplot.index
    ys = np.log(dftoplot['appearances'])
    btcvalue = dftoplot['BTC'].values
    data_normalizer = mp.colors.Normalize()
    color_map = mp.colors.LinearSegmentedColormap("my_map",\
        {'blue':[(0.0,  1.0, 1.0),(0.5,  0.5, 0.5),(1.0,  0.0, 0.0)],\
         'green': [(0.0,  0.0, 0.0),(0.25, 1.0, 1.0),(0.75, 0.0, 0.0),(1.0,  0.0, 0.0)],\
         'red':  [(0.0,  0.0, 0.0),(0.5,  0.5, 0.5),(1.0,  1.0, 1.0)]})
    N = len(xs)
    x_nums = np.arange(1, N+1)
    plt.bar(x_nums,ys,align="center",color = color_map(data_normalizer(btcvalue)))
    plt.xticks(x_nums, xs)
    return plt.figure(figsize=(200,50))

...and so we can explore the block signature for several values of $k$

In [ ]:
blocksignature(A,20)

In [ ]:
from ipywidgets import interact
    
interact(lambda k: blocksignature(A,k), k=(2,59))

## Conclusion: What could the data mean?

Let us now compare signatures for different blocks.

We preload a block from a "normal trading period"...

In [ ]:
block3 = pickle.load(open('stats_00000000000000000023a22bd73183e3c0bdc483c1bea6ea8ac5cfeb32e4d359','rb'))

...and visualize its signature.

In [ ]:
blocksignature(block3,20)

Now let's preload a block from a trading day during which the USD value of BTC was crashing...

In [ ]:
crashblock = pickle.load(open('stats_00000000000000000040881ea6d4e9232c42c20bbd67221ef866201dc285b567','rb'))

...and visualize its signature:

In [ ]:
blocksignature(crashblock,20)

And let us see them together, where we can vary the number of clusters:

In [ ]:
interact(lambda k: blocksignature(block3,k), k=(2,59))
interact(lambda k: blocksignature(crashblock,k), k=(2,59))

One can see, by inspection, a significant change in the signature of the blocks.

In particular, cluster 0, which roughly corresponds to the "funneling addresses" which have very few transactions and act primarily as "dummies" for the movement of BTC, has gone from red to blue.

It present some evidence that these signatures may contain a viable signal for USD/BTC price movements.